### Reference Resources
* http://nlp.seas.harvard.edu/2018/04/03/attention.html
* http://nlp.seas.harvard.edu/annotated-transformer/

### Code

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

### High Level Architecture

<center><div background="white"><img src = "http://nlp.seas.harvard.edu/images/the-annotated-transformer_14_0.png"></div></center>

In [3]:
class EncoderDecoder(nn.Module):

    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        
        ## super functions give access to methods of a parent class and returns a temporary object of the parent class
        ## in this case, we make sure that nn.Module's __init__ method is also called at the same time
        ## ie. assumption that nn.Module.__init__() has no argument
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):

        "Take in and process masked src and target sequences."

        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

class Generator(nn.Module):

    "Define standard linear + softmax generation step."
    
    def __init__(self, d_model, vocab):
        
        super().__init__()

        ## nn.Linear is a class that applies a linear transformation to the incoming data: y = xA^T + b
        ## ie. this is a function that projects the d_model dimensional vector to a vocab dimensional vector
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):

        ## F.log_softmax is a function that applies log_softmax to the input
        ## ie. we are applying log_softmax to the output of the linear transformation
        ## dim = -1 means that we are applying log_softmax to the last dimension. This follows circular indexing convention
        return F.log_softmax(self.proj(x), dim=-1)

### Encoder Module

**LayerNorm**
- Why do you need sefl.a_2 and self.b_2?

**SublayerConnection**
- Isn't the norm function duplicated? No, norm is applied in 2 places
    - The connection within each encoding sublayer (ie. between the multi-headed self-attention and the feed-forward layers)
    - When the signal is passed from the Encoder to the Decoder

In [14]:
## The encoder will have N=6 identical encoding sub-layers.

## Produce N identical layers
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

## Normalizing the distribution of the input
## https://arxiv.org/abs/1607.06450
class LayerNorm(nn.Module):
    
    def __init__(self, features, eps=1e-6):

        super().__init__()

        ## nn.Parameter is a special kind of Tensor that is automatically registered as a parameter when assigned as an attribute to a Module
        ## torch.ones creates a tensor of ones. features is a sequence of integers that define the dimensions of the output tensor
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    ## x is a tensor
    ## -1 specifies the last dimension; keepdim=True means that the output tensor has the same dimensionality, with the last dimension being 1
    ## Without keepdim=True, the output tensor would have one less dimension
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
## "Core encoder is a stack of N layers"
## Each "layer" is nn.Module with a callable forward(input, mask) method
## This is done through a __call__() method that is implemented by the nn.Module class

class Encoder(nn.Module):

    def __init__(self, layer, N):
        super().__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        
        ## "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [17]:
## Each encoding sub-layer consists 2 pieces: a mutli-head self-attention mechanism and a simple, position-wise fully connected feed-forward network
## The output of each encoding sub-layer will be passed through a dropout + residual + layernorm mechanism
##      The residual connection is a way of making the network deeper without increasing the risk of vanishing gradients
##      The layer normalization is a way of making the network deeper without increasing the risk of exploding gradients
##      Dropout is a regularization technique for reducing overfitting by preventing complex co-adaptations on training data


class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super().__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super().__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

### Decoder Module

**Decoder**
- Clarify the roles of <code>memory</code>, <code>src_mask</code>, and <code>tgt_mask </code>?

In [16]:
## This is also a 6-layer stack

class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super().__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [18]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super().__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))

        ## Notice that the src_attn is applied to the output of the self_attn
        ## This allows the encoder inputs to be incorporated after the decoder has processed its own inputs
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))

        return self.sublayer[2](x, self.feed_forward)

## Subsequent mask ensures that the decoder cannot see the future
## np.triu creates an upper triangular matrix from an array of ones, k=1 means starting from the kth diagonal
## torch.from_numpy converts a numpy array to a tensor, while == 0 flips the 1s to 0s and the 0s to 1s
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

### Attention Modules

Up to this point, we've only set up the scaffold of the model. Here, we build the mechanics behind a multi-headed attention module. We want to implement

<center><img src="http://nlp.seas.harvard.edu/images/the-annotated-transformer_33_0.png"></center>

$$\text{Attention}(Q,K,V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

The $\sqrt{d_k}$ scaling factor allows the softmax() to remain in reasonable range and not suffer vanishing gradients. Then, we expand this to multi-headed setting to capture multiple projections onto different representational subspaces of textual relations (via $W_i$). Ie.

$$\text{MultiHead}(Q,K,V) = \text{Concat}(\text{head}_0, \ldots , \text{head}_h) W^O$$

Where $\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$ and the projection matrices $W$ are of the dimensions 
- $W_i^Q \text{ and } W_i^K: d_{\text{model}} \times d_k$
- $W_i^V: d_m \times d_v$ 
- $W^O: hd_v \times d_{\text{model}}$

We assume that $d_v = d_k = d_{\text{model}} / h$ for simplicity

<center> <img src="http://nlp.seas.harvard.edu/images/the-annotated-transformer_38_0.png" height = 500> </center>

**def attention(...)**
- Why do you need to return <code>p_attn</code>?
- Why do you want to use the multi-headed architecture? Why not just pass the entire $d_model$ chunk in? And what does $d_model$ correspond to -- embedding dimensionality or length of memory?

In [19]:
## Scaled dot product attention
def attention(query, key, value, mask=None, dropout=None):

    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
             
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    
    p_attn = F.softmax(scores, dim = -1)

    if dropout is not None:
        p_attn = dropout(p_attn)
        
    return torch.matmul(p_attn, value), p_attn

In [21]:
class MultiHeadedAttention(nn.Module):

    def __init__(self, h, d_model, dropout=0.1):

        super().__init__()
        
        "Take in model size and number of heads."
        assert d_model % h == 0
        self.d_k = d_model // h ## the operator '//' means integer division
        self.h = h ## number of heads
        self.linears = clones(nn.Linear(d_model, d_model), 4) ## 4 linear projects account for Q, K, V, and the final output
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):

        if mask is not None:

            ## Same mask applied to all h heads.
            ## unsqueeze(i) adds a dimension of size 1 at the ith dimension
            ## https://pytorch.org/docs/stable/generated/torch.unsqueeze.html
            mask = mask.unsqueeze(1)
        
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        # Zip is a function that takes two lists and returns a list of tuples
        # .view(...) reshapes the tensor; -1 means infer the dimension from the other given dimension
        # .transpose(a,b) swaps the a and b dimensions
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        ## Note that self.attn is not used again. Why?
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)

        return self.linears[-1](x) # Output Tensor x W^O

### Position-wise feed-forward networks

Expresses 2 linear transformations with a ReLu in between
$$\text{FFN}(x) = \text{max}(0, xW_1 + b_1) W_2 + b_2$$

In [23]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

### Embeddings, Softmax and Positional Encoding


In [ ]:
class Embeddings(nn.Module):

    def __init__(self, d_model, vocab):

        super().__init__()

        ## num_embedding = vocab is the size of the vocabulary
        ## embedding_dim = d_model is the size of the embedding vector
        ## This is learning a matrix of V x d_model where every word in V is assigned a vector of size d_model
        ## d_model corresponds to the dimensionality of the semnatic space
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

The postionnal encoding used here are sin/cos values of the positions at different frequencies, ie.
$$ \text{PE}_{(\text{pos}, 2i)} = \text{sin}\left(\frac{\text{pos}}{1000^{2i/d_{\text{model}}}}\right)$$
$$ \text{PE}_{(\text{pos}, 2i+1)} = \text{cos}\left(\frac{\text{pos}}{1000^{2i/d_{\text{model}}}}\right)$$

**__init__**
- It's not clear to me why you need to do <code> pe.unsqueeze(0) </code>

In [ ]:
class PositionalEncoding(nn.Module):

    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)

        # torch.arrange creates a tensor of size max_len with values 0, 1 * step, 2 * step, ..., max_len-1, with step=1 by default
        position = torch.arange(0, max_len).unsqueeze(1)
        
        # this calculates the denominator of the positional encoding formula
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        
        
        # 0::2 means start at 0 and increment by 2 and 1::2 means start at 1 and increment by 2
        # For each row, every odd column is assigned a sin() value and every even column is assigned a cos() value
        # Each row corresponds to a single position in the sentence
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # why do we need to unsqueeze(0) here?
        # Every row of pe is now a single vector of size d_model
        pe = pe.unsqueeze(0)

        # register_buffer is used to buffer something that should not be considered a model parameter
        # this prevents the model from updating these values during training
        # buffer as opposed to parameter: https://discuss.pytorch.org/t/what-is-the-difference-between-register-buffer-and-register-parameter/2879
        self.register_buffer('pe', pe)
        
    def forward(self, x):

        # Variable is a wrapper around a tensor that supports autograd operations. This has been DEPRECATED
        # The autograd package provides automatic differentiation for all operations on Tensors.
        # requires_grade=False means that we do not want to compute gradients with respect to this variable
        # https://www.geeksforgeeks.org/variables-and-autograd-in-pytorch/
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)

        return self.dropout(x)

### Full Model

In [ ]:
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):

    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy

    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)

    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
            
    return model

In [ ]:
# Small example model.
tmp_model = make_model(10, 10, 2)

### Training

Label Smoothing
- What does this mean: "During training, we employed label smoothing of value $ϵ_{ls} = 0.1$. This hurts perplexity, as the model learns to be more unsure, but improves accuracy and BLEU score."

In [ ]:
## These handle the data batching

class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

In [ ]:
# the function loss_compute() will contain the logic for computing the loss and performing backpropagation
def run_epoch(data_iter, model, loss_compute):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.trg, 
                            batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens

In [ ]:
# This implements the Adam optimizer for stochastic gradient descent for high-dimensional data/parameters problems
## A method for efficient stochastic optimization that only requires first-order gradients with 
## little memory requirement. The method computes individual adaptive learning rates for different parameters 
## from estimates of first and second moments of the gradients; the name Adam is derived from adaptive moment estimation
## Some of Adam’s advantages are that the magnitudes of parameter updates are invariant to rescaling of the gradient,
## its stepsizes are approximately bounded by the stepsize hyperparameter, it does not require a stationary objective, 
## it works with sparse gradients, and it naturally performs a form of step size annealing.

# https://arxiv.org/pdf/1412.6980.pdf
# https://pytorch.org/docs/stable/optim.html

class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000, torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [ ]:
# label smoothing converts the output labels of the training set to a more even distribution of probabilities over the vocab
# Eg. if label index is "1" and the raw probability is [0, 1, 0], then convert it to [0.1, 0.8, 0.1] to express uncertainty
# https://towardsdatascience.com/label-smoothing-make-your-model-less-over-confident-b12ea6f81a9a
# https://arxiv.org/pdf/1512.00567.pdf

class LabelSmoothing(nn.Module):

    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super().__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

### Example: Copying Task

Given a random set of input symbols from a small vocab, the goal is to generate back those symbols.

In [ ]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)

        # criterion is the loss function and returns a tensor
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)) / norm

        # Backpropagation
        
        # .backward() computes the d(loss)/dx for every parameter x which has requires_grad=True.
        # These are accumulated into x.grad for every parameter x.
        loss.backward()

        # this updates the parameters
        if self.opt is not None:

            # update the parameters and rates
            self.opt.step() 

            # .zero_grad() clears x.grad for every parameter x in the model that has already been updated with step()
            # https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
            self.opt.optimizer.zero_grad() 

        return loss.data[0] * norm

In [ ]:
# Synthetic Data
def data_gen(V, batch, nbatches):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, 10)))
        data[:, 0] = 1

# Train the simple copy task.
V = 11
criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
model = make_model(V, V, N=2)
model_opt = NoamOpt(model.src_embed[0].d_model, 1, 400, torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

for epoch in range(10):
    model.train()
    run_epoch(data_gen(V, 30, 20), model, SimpleLossCompute(model.generator, criterion, model_opt))
    model.eval()
    print(run_epoch(data_gen(V, 30, 5), model, SimpleLossCompute(model.generator, criterion, None)))

### Appendix

In [13]:
## Syntax note: __call__ is embedded in the nn.Module class to make the objects callable with forward(*args)
class Test():
    def __init__(self, x):
        self.x = x
    def __call__(self, *args):
        return self.forward(*args)
    def forward(self, y, z=0):
        return self.x + y + z 
t = Test(5)
t(2) ## prints 7
t(2, 3) ## prints 10

10